In [83]:
"""
Detecting duplicate quora questions
feature engineering
@author: Abhishek Thakur
"""

import cPickle
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')

104904it [00:39, 2629.17it/s]

In [79]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())


data = pd.read_csv('data/cor_train.csv', sep=',')
data = data.drop(['id', 'qid1', 'qid2'], axis=1)


model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)
norm_model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)
norm_model.init_sims(replace=True)


In [87]:
print data.question1.values

['What is the step by step guide to invest in share market in india?'
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?'
 'How can I increase the speed of my internet connection while using a VPN?'
 ..., 'What is one coin?'
 'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?'
 'What is like to have sex with cousin?']


In [88]:
question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0

for i, q in enumerate(tqdm_notebook(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in enumerate(tqdm_notebook(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))
/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:437: RuntimeWarning: invalid value encountered in double_scalars
  / np.double(np.bitwise_or(u != 0, v != 0).sum()))
/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:631: RuntimeWarning: invalid value encountered in double_scalars
  return abs(u - v).sum() / abs(u + v).sum()


In [98]:
print data.shape
print data.ix[:,3:].columns

(404290, 14)
Index([u'cosine_distance', u'cityblock_distance', u'jaccard_distance',
       u'canberra_distance', u'euclidean_distance', u'minkowski_distance',
       u'braycurtis_distance', u'skew_q1vec', u'skew_q2vec', u'kur_q1vec',
       u'kur_q2vec'],
      dtype='object')


In [95]:
cPickle.dump(question1_vectors, open('data/q1_w2v.pkl', 'wb'), -1)
cPickle.dump(question2_vectors, open('data/q2_w2v.pkl', 'wb'), -1)
data.ix[:,3:].to_csv('data/trainset_dis.csv', index=False)

In [104]:
data.ix[:,3:]

,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,6.897204e-02,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301
1,5.121644e-01,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444
2,2.220086e-01,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131
3,6.504112e-01,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568
4,3.699935e-01,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876
5,2.752421e-01,10.254048,1.0,150.335589,0.741946,0.337470,0.400061,0.102946,-0.077741,-0.335505,-0.057847
6,8.206374e-01,17.414754,1.0,200.598725,1.281122,0.597082,0.816676,-0.090366,-0.000266,0.296892,0.130717
7,6.461523e-02,5.008762,1.0,95.395891,0.359486,0.161125,0.187150,0.346824,0.368469,0.635773,0.865446
8,4.440892e-16,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.202336,0.202336,-0.434927,-0.434927
9,2.847095e-01,10.634987,1.0,153.647611,0.754599,0.337939,0.409126,0.120050,0.204449,-0.214573,-0.126814


In [107]:
dis_train=pd.read_csv('data/trainset_dis.csv')
dis_train

,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,6.897204e-02,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301
1,5.121644e-01,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444
2,2.220086e-01,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131
3,6.504112e-01,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568
4,3.699935e-01,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876
5,2.752421e-01,10.254048,1.0,150.335589,0.741946,0.337470,0.400061,0.102946,-0.077741,-0.335505,-0.057847
6,8.206374e-01,17.414754,1.0,200.598725,1.281122,0.597082,0.816676,-0.090366,-0.000266,0.296892,0.130717
7,6.461523e-02,5.008762,1.0,95.395891,0.359486,0.161125,0.187150,0.346824,0.368469,0.635773,0.865446
8,4.440892e-16,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.202336,0.202336,-0.434927,-0.434927
9,2.847095e-01,10.634987,1.0,153.647611,0.754599,0.337939,0.409126,0.120050,0.204449,-0.214573,-0.126814
